In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

In [3]:
df = pd.read_csv("merged_table_23-24_with_null.csv", encoding="ISO-8859-1")

In [4]:
df['time'] = pd.to_datetime(df['time'], format='%m/%d/%y %H:%M')

In [5]:
# Iterate over the columns with object data type and apply the transformations
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].str.replace(r'\s+', '', regex=True)
    df[column] = df[column].replace('', np.nan)
    df[column] = df[column].astype(float)

In [6]:
print(df.isnull().sum())

time                   0
ni_in               4803
fe_in               4802
sio2_in             4802
cao_in              4802
mgo_in              4802
al2o3_in            4802
fe_ni               4803
s_m                 4802
bc                  4802
loi_in              8782
mc_kilnfeed            1
fc_coal             6624
gcv_coal            6624
voltage_pry          779
voltage_sec          510
current_pry          750
current_sec1         503
current_sec2         503
current_sec3         503
load                 498
power_factor         811
realisasi_beban      498
rpm                  745
kg_tco              5036
charge_kiln          256
tdo                    1
pry_p                256
sec_p                256
pry_v                256
sec_v                256
total_coal           284
total_fuel          4934
a_f_ratio           2100
reductor_consume     743
t_tic162            1112
t_tic163            1112
metal_temp          1179
ni_met              1177
c_met               1177


In [7]:
# Define the time ranges
start_range1 = pd.to_datetime('2024-07-31 01:00 AM')
end_range1 = pd.to_datetime('2024-07-31 03:00 PM')
start_range2 = pd.to_datetime('2024-08-01 12:00 AM')
end_range2 = pd.to_datetime('2024-11-12 11:00 PM')

# Filter the DataFrame to include only the specified time ranges
intersection_df = df[((df['time'] >= start_range1) & (df['time'] <= end_range1)) | ((df['time'] >= start_range2) & (df['time'] <= end_range2))]

# Display the filtered DataFrame
print(intersection_df.shape)

(2511, 48)


In [8]:
# Save the filtered data to a new CSV file (optional)
intersection_df.to_csv('filtered_intersection_df.csv', index=False)

In [9]:
print(intersection_df.isnull().sum())

time                   0
ni_in                  0
fe_in                  0
sio2_in                0
cao_in                 0
mgo_in                 0
al2o3_in               0
fe_ni                  0
s_m                    0
bc                     0
loi_in              2511
mc_kilnfeed            1
fc_coal             1815
gcv_coal            1815
voltage_pry            1
voltage_sec            1
current_pry            2
current_sec1           1
current_sec2           1
current_sec3           1
load                   0
power_factor           3
realisasi_beban        0
rpm                    0
kg_tco                23
charge_kiln            8
tdo                    1
pry_p                  8
sec_p                  8
pry_v                  8
sec_v                  8
total_coal             8
total_fuel             8
a_f_ratio            124
reductor_consume       0
t_tic162               0
t_tic163               0
metal_temp             3
ni_met                 1
c_met                  1


In [10]:
input_cols = intersection_df.columns[intersection_df.columns.get_loc('ni_in'):intersection_df.columns.get_loc('t_tic163') + 1]
output_cols = intersection_df.columns[intersection_df.columns.get_loc('metal_temp'):intersection_df.columns.get_loc('loi_kalsin') + 1]
smelting_cols = intersection_df.columns[intersection_df.columns.get_loc('voltage_pry'):intersection_df.columns.get_loc('realisasi_beban') + 1]

In [11]:
intersection_df[output_cols] = intersection_df[output_cols].shift(-16)
intersection_df = intersection_df.iloc[:-16]

# Reset the index
intersection_df.reset_index(drop=True, inplace=True)

C:\Users\rianr\AppData\Local\Temp\ipykernel_29256\3846504361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersection_df[output_cols] = intersection_df[output_cols].shift(-16)


In [12]:
filtered_intersection_df = intersection_df.drop(columns=['loi_in', 'fe_ni', 'sio2_in', 'mgo_in', 'voltage_pry', 'voltage_sec'])
input_cols = input_cols.drop(['loi_in', 'fe_ni', 'sio2_in', 'mgo_in', 'voltage_pry', 'voltage_sec'])

In [13]:
print(filtered_intersection_df.isnull().sum())

time                   0
ni_in                  0
fe_in                  0
cao_in                 0
al2o3_in               0
s_m                    0
bc                     0
mc_kilnfeed            1
fc_coal             1815
gcv_coal            1815
current_pry            2
current_sec1           1
current_sec2           1
current_sec3           1
load                   0
power_factor           3
realisasi_beban        0
rpm                    0
kg_tco                22
charge_kiln            8
tdo                    1
pry_p                  8
sec_p                  8
pry_v                  8
sec_v                  8
total_coal             8
total_fuel             8
a_f_ratio            123
reductor_consume       0
t_tic162               0
t_tic163               0
metal_temp             3
ni_met                 1
c_met                  1
si_met                 1
fe_met                 1
s_met                  1
ni_slag                3
fe_slag                3
t_kalsin               0


In [14]:
def interpolate(df, column):
    # Convert the index to numerical values for interpolation
    x = np.arange(len(df))
    y = df[column].values

    # Identify the indices of the non-null values
    non_null_indices = np.where(~np.isnan(y))[0]

    # Perform polynomial interpolation using scipy's interp1d
    poly_interp = interp1d(non_null_indices, y[non_null_indices], kind='cubic', fill_value="extrapolate")

    # Interpolate the null values
    y_interp = poly_interp(x)

    # # Create a single figure with two subplots
    # plt.figure(figsize=(12, 4))

    # # Plot the original data
    # plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
    # plt.plot(df['time'], y, marker='o', linestyle='-', label='Original Data')
    # plt.title('Original Data: ' + column)
    # plt.xlabel('Time')
    # plt.ylabel(column)
    # plt.grid(True)
    # plt.legend()

    # # Plot the interpolated data
    # plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
    # plt.plot(df['time'], y, 'o', label='Original Data')
    # plt.plot(df['time'], y_interp, '-', label='Interpolated Data')
    # plt.title('Interpolated Data: ' + column)
    # plt.xlabel('Time')
    # plt.ylabel(column)
    # plt.grid(True)
    # plt.legend()

    # # Display the plots
    # plt.tight_layout()
    # plt.show()

    # Update the DataFrame with the interpolated values
    df[column] = y_interp

In [15]:
for column in filtered_intersection_df.columns:
    if filtered_intersection_df[column].isnull().any():
        interpolate(filtered_intersection_df, column)

### Result

In [16]:
print(filtered_intersection_df.isnull().sum())
print(filtered_intersection_df.shape)

time                0
ni_in               0
fe_in               0
cao_in              0
al2o3_in            0
s_m                 0
bc                  0
mc_kilnfeed         0
fc_coal             0
gcv_coal            0
current_pry         0
current_sec1        0
current_sec2        0
current_sec3        0
load                0
power_factor        0
realisasi_beban     0
rpm                 0
kg_tco              0
charge_kiln         0
tdo                 0
pry_p               0
sec_p               0
pry_v               0
sec_v               0
total_coal          0
total_fuel          0
a_f_ratio           0
reductor_consume    0
t_tic162            0
t_tic163            0
metal_temp          0
ni_met              0
c_met               0
si_met              0
fe_met              0
s_met               0
ni_slag             0
fe_slag             0
t_kalsin            0
pic_161             0
loi_kalsin          0
dtype: int64
(2495, 42)


In [17]:
filtered_intersection_df.to_excel('filtered_intersection_df_interpolated.xlsx', index=False)

In [18]:
min_max_df = filtered_intersection_df.agg(['min', 'max']).transpose()

# Display the min and max values for each column
print(min_max_df)

                                  min                  max
time              2024-07-31 01:00:00  2024-11-12 07:00:00
ni_in                            1.33                 2.09
fe_in                           11.95                23.62
cao_in                           0.36                  3.5
al2o3_in                          1.0                 4.24
s_m                          1.471361             3.646552
bc                           0.292244             0.694697
mc_kilnfeed                     12.22                29.16
fc_coal                          31.9                 80.0
gcv_coal                       5823.4              6987.85
current_pry                      -0.0               221.97
current_sec1                     -0.0                197.0
current_sec2                     -0.0                229.0
current_sec3                    -14.8                282.2
load                            -22.4                94.82
power_factor                     -1.0                  1

In [19]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [20]:
# Remove outliers from 'total_coal' and 'total_fuel'
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'total_coal')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'total_fuel')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'charge_kiln')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'kg_tco')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'tdo')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_pry')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_sec1')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_sec2')
filtered_intersection_df = remove_outliers_iqr(filtered_intersection_df, 'current_sec3')

# Display the shape of the DataFrame after removing outliers
print(filtered_intersection_df.shape)

(1564, 42)


In [21]:
min_max_df = filtered_intersection_df.agg(['min', 'max']).transpose()

# Display the min and max values for each column
print(min_max_df)

                                  min                  max
time              2024-07-31 01:00:00  2024-11-12 07:00:00
ni_in                            1.34                 2.09
fe_in                           11.95                23.62
cao_in                           0.36                  3.5
al2o3_in                          1.0                 3.53
s_m                          1.471361             3.646552
bc                           0.292244             0.694697
mc_kilnfeed                     12.22                29.16
fc_coal                          31.9                 80.0
gcv_coal                       5823.4              6987.85
current_pry                     20.48                23.59
current_sec1                     16.8                 25.8
current_sec2                     17.0                 26.0
current_sec3                     16.9                 25.9
load                            -22.4                94.82
power_factor                     -1.0                 0.

## Model Training

In [22]:
data_df = filtered_intersection_df.copy()
print(data_df.shape)

(1564, 42)


In [23]:
# Initialize the RobustScaler
scaler = RobustScaler()

# Apply Robust Scaling to the DataFrame
scaled_data = scaler.fit_transform(data_df.drop(columns=['time']))

# Convert the scaled data back to a DataFrame
scaled_data_df = pd.DataFrame(scaled_data, columns=data_df.drop(columns=['time']).columns)

In [24]:
input_cols = scaled_data_df.columns[scaled_data_df.columns.get_loc('ni_in'):scaled_data_df.columns.get_loc('t_tic163') + 1]
output_cols = scaled_data_df.columns[scaled_data_df.columns.get_loc('metal_temp'):scaled_data_df.columns.get_loc('loi_kalsin') + 1]

In [25]:
# Define split configurations
split_ratios = [0.1]

# Initialize models
models = {
    'XGBoost': XGBRegressor(),
}

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='uniform_average')  # Average across outputs
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='uniform_average')
    return mse, rmse, mae

# Iterate over split ratios
for test_size in split_ratios:
    print(f"Split ratio: {int((1 - test_size) * 100)}:{int(test_size * 100)}")
    
    # Split the data
    X = scaled_data_df[input_cols]
    y = scaled_data_df[output_cols]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    y_train = y_train.values
    y_test = y_test.values

    # Initialize results for this split ratio
    results_mse, results_rmse, results_mae = [], [], []
    
    # Train each model
    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Inverse transform y_test and y_pred to original scale
        y_test_original = scaler.inverse_transform(np.hstack([X_test, y_test]))[:, len(input_cols):]
        y_pred_original = scaler.inverse_transform(np.hstack([X_test, y_pred]))[:, len(input_cols):]
        
        # Calculate metrics for each target
        for col_idx, col_name in enumerate(output_cols):
            mse = mean_squared_error(y_test_original[:, col_idx], y_pred_original[:, col_idx])
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test_original[:, col_idx], y_pred_original[:, col_idx])
            
            # Append results
            results_mse.append([name, col_name, mse])
            results_rmse.append([name, col_name, rmse])
            results_mae.append([name, col_name, mae])

        if name == 'XGBoost':
            # Save the XGBoost model in pkl format
            with open(f'xgboost_model_{int((1 - test_size) * 100)}_{int(test_size * 100)}.pkl', 'wb') as file:
                pickle.dump(model, file)
    
    # Create DataFrames for metrics
    mse_df = pd.DataFrame(results_mse, columns=['Model', 'Feature', 'MSE'])
    rmse_df = pd.DataFrame(results_rmse, columns=['Model', 'Feature', 'RMSE'])
    mae_df = pd.DataFrame(results_mae, columns=['Model', 'Feature', 'MAE'])

    # Pivot DataFrames
    mse_pivot = mse_df.pivot(index='Feature', columns='Model', values='MSE')
    rmse_pivot = rmse_df.pivot(index='Feature', columns='Model', values='RMSE')
    mae_pivot = mae_df.pivot(index='Feature', columns='Model', values='MAE')

    # Display results
    print("MSE Table:")
    print(mse_pivot)
    print("\nRMSE Table:")
    print(rmse_pivot)
    print("\nMAE Table:")
    print(mae_pivot)

    # Save results for this split ratio
    filename = f'xgb_model_performance_metrics_{int((1 - test_size) * 100)}_{int(test_size * 100)}.xlsx'
    with pd.ExcelWriter(filename) as writer:
        mse_pivot.to_excel(writer, sheet_name='MSE')
        rmse_pivot.to_excel(writer, sheet_name='RMSE')
        mae_pivot.to_excel(writer, sheet_name='MAE')

Split ratio: 90:10
Training XGBoost...


MSE Table:
Model           XGBoost
Feature                
c_met          0.003208
fe_met         0.022630
fe_slag        0.143248
loi_kalsin     0.032028
metal_temp    67.789136
ni_met         0.060521
ni_slag        0.000021
pic_161        1.800694
s_met          0.006027
si_met         0.002547
t_kalsin    1440.409318

RMSE Table:
Model         XGBoost
Feature              
c_met        0.056643
fe_met       0.150432
fe_slag      0.378482
loi_kalsin   0.178964
metal_temp   8.233416
ni_met       0.246010
ni_slag      0.004550
pic_161      1.341899
s_met        0.077636
si_met       0.050467
t_kalsin    37.952725

MAE Table:
Model         XGBoost
Feature              
c_met        0.040135
fe_met       0.105521
fe_slag      0.255039
loi_kalsin   0.105510
metal_temp   5.384298
ni_met       0.130421
ni_slag      0.003283
pic_161      0.799523
s_met        0.033909
si_met       0.035015
t_kalsin    27.485651


## Optimization

In [36]:
import numpy as np
import pandas as pd
import pickle

# ----------------------------
# 1. Load the pre-trained XGBoost model
# ----------------------------
with open("xgboost_model_90_10.pkl", "rb") as f:
    xgb_model = pickle.load(f)

# ----------------------------
# 2. Prepare your dataset
# ----------------------------
df = filtered_intersection_df.copy()
if "time" in df.columns:
    df.drop(columns=["time"], inplace=True)

exact_columns = [
    "ni_in", "fe_in", "cao_in", "al2o3_in", "s_m", "bc", "mc_kilnfeed",
    "fc_coal", "gcv_coal", "kg_tco", "charge_kiln", "tdo"
]

positive_columns = [
    "current_pry", "current_sec1", "current_sec2", "current_sec3", "load",
    "power_factor", "realisasi_beban", "rpm", "pry_p", "sec_p", "pry_v", "sec_v",
    "total_coal", "total_fuel", "a_f_ratio", "reductor_consume", "t_tic162", "t_tic163"
]

# XGBoost input features (must match the columns the model was trained on)
xgb_input_features = [
    "ni_in", "fe_in", "cao_in", "al2o3_in", "s_m", "bc", "mc_kilnfeed",
    "fc_coal", "gcv_coal", "current_pry", "current_sec1", "current_sec2",
    "current_sec3", "load", "power_factor", "realisasi_beban", "rpm",
    "kg_tco", "charge_kiln", "tdo", "pry_p", "sec_p", "pry_v", "sec_v",
    "total_coal", "total_fuel", "a_f_ratio", "reductor_consume",
    "t_tic162", "t_tic163"
]

# Choose one row to keep exact columns fixed
base_row = df.sample(n=1, random_state=42).iloc[0]

# ----------------------------
# 3. Define the optimization problem using pymoo, 
#    using dataset min/max for the decision variable bounds
# ----------------------------
from pymoo.core.problem import Problem

class MyProblem(Problem):
    def __init__(self, xgb_model, df, base_row, exact_cols, pos_cols, input_features):
        self.xgb_model = xgb_model
        self.df = df
        self.base_row = base_row
        self.exact_cols = exact_cols
        self.pos_cols = pos_cols
        self.input_features = input_features
        
        # Number of decision variables = length of pos_cols
        n_var = len(pos_cols)
        n_obj = 1  # Minimizing total_coal + total_fuel
        
        # Constraints for outputs: 11 basic + 2 more for c_met and 2 for si_met = 13 total
        n_constr = 13
        
        # Build lower (xl) and upper (xu) bounds from dataset min/max for each pos_col
        xl = []
        xu = []
        for col_name in pos_cols:
            col_min = df[col_name].min()
            col_max = df[col_name].max()
            xl.append(col_min)
            xu.append(col_max)

        super().__init__(n_var=n_var, n_obj=n_obj, n_constr=n_constr,
                         xl=np.array(xl), xu=np.array(xu))

    def _evaluate(self, X, out, *args, **kwargs):
        pop_size = X.shape[0]
        F = np.zeros(pop_size)
        G = np.zeros((pop_size, self.n_constr))

        for i in range(pop_size):
            row_data = self.base_row.copy()
            
            # Fill in variable columns with the candidate solution
            for col_idx, col_name in enumerate(self.pos_cols):
                row_data[col_name] = X[i, col_idx]

            # Objective: total_coal + total_fuel
            F[i] = row_data["total_coal"] + row_data["total_fuel"]

            # Prepare the input for model prediction
            input_df = pd.DataFrame([row_data], columns=self.input_features)
            preds = self.xgb_model.predict(input_df)

            # Extract predictions (assuming shape (1, 11))
            metal_temp   = preds[0][0]
            ni_met       = preds[0][1]
            c_met        = preds[0][2]
            si_met       = preds[0][3]
            fe_met       = preds[0][4]
            s_met        = preds[0][5]
            ni_slag      = preds[0][6]
            fe_slag      = preds[0][7]
            t_kalsin     = preds[0][8]
            pic_161      = preds[0][9]
            loi_kalsin   = preds[0][10]

            # Constraints
            G[i, 0]  = metal_temp - 1300      # metal_temp > 1300
            G[i, 1]  = ni_met - 17           # ni_met > 17
            G[i, 2]  = c_met - 1             # c_met > 1
            G[i, 3]  = 3 - c_met             # c_met < 3
            G[i, 4]  = si_met - 1            # si_met > 1
            G[i, 5]  = 2 - si_met            # si_met < 2
            G[i, 6]  = fe_met                # fe_met > 0
            G[i, 7]  = 0.4 - s_met           # s_met < 0.4
            G[i, 8]  = ni_slag               # ni_slag > 0
            G[i, 9]  = fe_slag               # fe_slag > 0
            G[i, 10] = t_kalsin - 600        # t_kalsin > 600
            G[i, 11] = pic_161               # pic_161 > 0
            G[i, 12] = 1 - loi_kalsin        # loi_kalsin < 1

        out["F"] = F
        out["G"] = G

# Instantiate the problem with dataset min/max bounds
problem = MyProblem(
    xgb_model=xgb_model,
    df=df,
    base_row=base_row,
    exact_cols=exact_columns,
    pos_cols=positive_columns,
    input_features=xgb_input_features
)

# ----------------------------
# 4. Run the genetic algorithm
# ----------------------------
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

algorithm = GA(pop_size=50)

res = minimize(problem, algorithm, ("n_gen", 50), seed=1, verbose=True)

# ----------------------------
# 5. Retrieve and display results with fallback
# ----------------------------
def print_best_individual_info(best_vars_array):
    best_row_data = base_row.copy()
    for idx, col in enumerate(positive_columns):
        best_row_data[col] = best_vars_array[idx]

    print("\nBest individual's full row:")
    print(best_row_data)

    df_min = df.min()
    df_max = df.max()
    print("\nEach column value with min/max from dataset:")
    for col in best_row_data.index:
        print(f"{col}: {best_row_data[col]} (Min: {df_min[col]}, Max: {df_max[col]})")

if res.X is None or res.F is None:
    print("Optimization did not produce a final result. Retrieving best individual from population.")
    if hasattr(res, "pop") and res.pop is not None:
        pop_obj_vals = res.pop.get("F")
        best_idx = np.argmin(pop_obj_vals)
        best_vars = res.pop.get("X")[best_idx]
        best_f = pop_obj_vals[best_idx]
        print("Best known individual's decision variables:", best_vars)
        print("Objective (total_coal + total_fuel):", best_f)
        print_best_individual_info(best_vars)
    else:
        print("No population info available.")
else:
    best_vars = res.X
    best_f = res.F[0]
    print("Best decision variable values:", best_vars)
    print("Minimum total_coal + total_fuel:", best_f)
    print_best_individual_info(best_vars)

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       50 |  4.6458606720 |  4.7476741032 |             - |             -
     2 |      100 |  4.6437437087 |  4.6889061659 |             - |             -
     3 |      150 |  4.6297383904 |  4.6547905242 |             - |             -
     4 |      200 |  4.6232888252 |  4.6452166152 |             - |             -
     5 |      250 |  4.6232888252 |  4.6366691872 |             - |             -
     6 |      300 |  4.6232888252 |  4.6270233443 |             - |             -
     7 |      350 |  4.5917390585 |  4.6231029281 |             - |             -
     8 |      400 |  4.5917390585 |  4.6215774664 |             - |             -
     9 |      450 |  4.5917390585 |  4.6199186161 |             - |             -
    10 |      500 |  4.5816587955 |  4.6149643072 |             - |             -
    11 |      550 |  4.5816587955 |  4.6076613373 |             - |             -
    12 |      60